<a href="https://colab.research.google.com/github/NSCC-HydroPandas/Canadian-Tidal-Data-Metadat/blob/main/Downloading_Monthly_Means_by_Station.ipynb" target="_parent">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Canadian Tide Monthly Means Data Downloading

**1. Set up Enviroment**

In [ ]:
!pip install requests
!pip install pandas
import requests
import pandas as pd
import json
import time
from urllib.parse import urlparse, parse_qs

**2. User Input**

The script will ask for the follow:
- Station Number
- Start Year
- End Year

In [ ]:
stn_num = int(input('Please Provide Station Number: '))
start_date = input('Provide Start Date in YYYY Format: ')
end_date = input('Provide End Date in YYYY Format: ')

**4. Find station endpoint based on station number provided**

In [ ]:
# Correcting to have Station in 5 digit format
stn_num = f"{stn_num:05d}"
stn_url = f'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations?code={stn_num}'
stn_url

In [ ]:
df = pd.read_json(stn_url)
stn_id = df['id'].to_list()
stn_id

**5. Create List of url for API requests**

In [ ]:
request_list = []

i = 0
for year in range(int(start_date),int(end_date)+1):
    year_str = str(year)
    for month in range(1,13):
        month_str = f"{month:02d}"
        data_url = (f'https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations/{stn_id[0]}/stats/calculate-monthly-mean?year={year_str}&month={month_str}&timeZone=UTC')
        request_list.append(data_url)



In [ ]:
request_list

**6. Request Montly Means using the URL request list From the Public API**

In [ ]:
r = requests.get('https://api.iwls-sine.azure.cloud-nuage.dfo-mpo.gc.ca/api/v1/stations/5cebf1e33d0f4a073c4bc21f/stats/calculate-monthly-mean?year=2022&month=02&timeZone=UTC')



In [ ]:
data = pd.DataFrame()


for url in request_list:
    parsed = urlparse(url)
    params = parse_qs(parsed.query)
    year = int(params['year'][0])
    month = int(params['month'][0])
    try:
        r = requests.get(url)
        d = json.loads(r.text)
        df = pd.DataFrame([d])
        df['year'] = year
        df['month'] = f"{month:02d}"
        data = pd.concat([data, df], ignore_index=True)
        # time.sleep(1)
    except Exception as e:
        print(f'No Monthly mean for {year}-{month:02d} due insignificant amount of obervation data that month')
        continue

# Save CSV
data.to_csv(f'{stn_num}_{start_date}-01_{end_date}-12.csv', index=False)
